<a href="https://colab.research.google.com/github/sn00wden/Ball_detection_pool/blob/main/Ball_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from __future__ import annotations


from PIL import Image 
from torch.nn import Conv2d, Linear, MaxPool2d, Sequential, ReLU 
from torch.utils.data import DataLoader, Dataset

import numpy as np
import os,os.path
import pandas as pd
import torch
import torchvision.transforms as T


# Data include and cleaning

In [ ]:
img = Image.open("./images/0.png")
coords = pd.read_csv("./coords.csv")
img_width = img.size
print(img_width)
# print(type(coords))

(100, 56)


In [ ]:
class PoolImageDataset(Dataset):
  global x_min,x_max,y_min,y_max

  #Dimensions of my pool
  width , height = 1.900, 0.956
  x_min,x_max = -(width/2),width/2
  y_min,y_max = -(height/2),height/2
  

  def __init__(
      self,
      csv_file_path : str = './coords.csv',
      data_path : str = "./images/",
      transforms : T.Compose = None
      ) -> None:

    super().__init__()
    self.labels = pd.read_csv(csv_file_path)
    self.images_path = data_path
    self.transforms = transforms
    # mapping and translating the x and y
    self.image_width, self.image_length = Image.open(data_path+"0.png").size
    self.clean_values()

  @staticmethod
  def mapping_and_translation(
      value : float,
      actual_ref_min : float,
      actual_ref_max : float,
      offset : int # offset = translation quantity
      ) -> int : 
      new_ref_max = offset
      new_ref_min = -offset
      new_value = int(((value - actual_ref_min) * (new_ref_max - new_ref_min)/ (actual_ref_max - actual_ref_min) + new_ref_min)+0.5)
      return new_value + offset
  
  def clean_values(self):
    self.labels.drop('z', inplace = True,axis=1)
    self.labels.drop(self.labels[(self.labels['x']>=x_max) | (self.labels['x'] <= x_min) ].index, inplace=True) #
    self.labels.drop(self.labels[(self.labels.y >= y_max) | (self.labels.y <= y_min) ].index, inplace=True) #

    self.labels["x"] = self.labels["x"].apply(lambda x : self.mapping_and_translation(x,x_min,x_max,int(self.image_width/2)))
    self.labels["y"] = self.labels["y"].apply(lambda y : self.mapping_and_translation(y,y_min,y_max,int(self.image_length/2)))
    #change coords if the translation changed
    


  def __len__(self):
    return len(os.listdir(self.images_path))
  
  def __getitem__(self, idx) -> dict:
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.images_path,
                                 f"{str(idx)}.png")
        image = Image.open(img_name)
        # print(self.labels[self.labels.id == idx])
        coords = self.labels[self.labels.id == idx].iloc[:]
        coords = np.array(coords)
        # print(coords)
        # coords = coords[:-1:].astype('float').reshape(-1)
        sample = {'image': image, 'coords': coords}

        print(self.transforms(sample['image']))
        # print(type(sample))

        return sample

In [ ]:
os.listdir("./images/")

In [ ]:
transforms = T.Compose([T.ToTensor(),T.RandAugment()])

In [ ]:
dataset = PoolImageDataset(transforms = transforms)

In [ ]:
dataset[1]
# list(enumerate(dataset))

TypeError: ignored